# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [8]:
import numpy as np
import pandas as pd
import requests

# 1) Carregar a base, contar linhas e contar linhas não-duplicadas
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(f'Número de linhas: {sinasc.shape[0]}.')
num_nd = sinasc.drop_duplicates().shape[0]
print(f'Número de linhas não duplicadas: {num_nd}.')
# Não há duplicados
sinasc.head(10)

Número de linhas: 27028.
Número de linhas não duplicadas: 27028.


,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
5,1,2516500.0,110001,1,33,5.0,1 a 3 anos,621005.0,2.0,1.0,...,5,6,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
6,1,2516500.0,110001,1,37,1.0,8 a 11 anos,999992.0,1.0,0.0,...,5,7,ATIVO,MUNIC,Novo Horizonte do Oeste,Rondônia,-11.70974,-61.99439,230.0,843.446
7,1,2516500.0,110001,1,27,2.0,12 anos ou mais,514120.0,0.0,0.0,...,5,8,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
8,1,2679477.0,110001,1,20,5.0,8 a 11 anos,621005.0,0.0,1.0,...,2,9,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
9,1,2679477.0,110001,1,30,2.0,1 a 3 anos,999992.0,1.0,1.0,...,5,10,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [10]:
# 2) Contar o número de missing por variável
soma_missing = sinasc.isna().sum().to_dict()
soma_missing

{'ORIGEM': 0,
 'CODESTAB': 115,
 'CODMUNNASC': 0,
 'LOCNASC': 0,
 'IDADEMAE': 0,
 'ESTCIVMAE': 317,
 'ESCMAE': 312,
 'CODOCUPMAE': 2907,
 'QTDFILVIVO': 1573,
 'QTDFILMORT': 2098,
 'CODMUNRES': 0,
 'GESTACAO': 1232,
 'GRAVIDEZ': 79,
 'PARTO': 49,
 'CONSULTAS': 0,
 'DTNASC': 0,
 'HORANASC': 21,
 'SEXO': 4,
 'APGAR1': 96,
 'APGAR5': 103,
 'RACACOR': 647,
 'PESO': 0,
 'IDANOMAL': 591,
 'DTCADASTRO': 0,
 'CODANOMAL': 26814,
 'NUMEROLOTE': 0,
 'VERSAOSIST': 0,
 'DTRECEBIM': 0,
 'DIFDATA': 0,
 'DTRECORIGA': 27028,
 'NATURALMAE': 298,
 'CODMUNNATU': 298,
 'CODUFNATU': 298,
 'ESCMAE2010': 249,
 'SERIESCMAE': 12710,
 'DTNASCMAE': 40,
 'RACACORMAE': 661,
 'QTDGESTANT': 1212,
 'QTDPARTNOR': 1879,
 'QTDPARTCES': 1791,
 'IDADEPAI': 19421,
 'DTULTMENST': 10072,
 'SEMAGESTAC': 1238,
 'TPMETESTIM': 1238,
 'CONSPRENAT': 930,
 'MESPRENAT': 2867,
 'TPAPRESENT': 265,
 'STTRABPART': 947,
 'STCESPARTO': 747,
 'TPNASCASSI': 61,
 'TPFUNCRESP': 67,
 'TPDOCRESP': 14,
 'DTDECLARAC': 52,
 'ESCMAEAGR1': 249,
 'STDN

In [34]:
# 3) Contar o número de missing de apenas algumas colunas
sinasc_cut = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
print(sinasc_cut.isna().sum())
sinasc_cut.head(10)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0


In [77]:
# 4) Remover todos os registros com Apgar5 não preenchido. Contar novamente o número de linhas e o número de missings.
sinasc_edit = sinasc_cut[sinasc_cut['APGAR5'].notna()]
print(f'Número de linhas: {sinasc_edit.shape[0]}.')
num_miss = sinasc_edit['APGAR5'].isna().sum()
print(f'Número de missing: {num_miss}.')
sinasc_edit.head(10)

Número de linhas: 26925.
Número de missing: 0.


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0


In [85]:
# 5) Alterar NaN para 9 em duas colunas
# sinasc_edit_a[['ESTCIVMAE', 'CONSULTAS']] = sinasc_edit[['ESTCIVMAE', 'CONSULTAS']].fillna('9')
sinasc_edit_a = sinasc_edit.fillna({'ESTCIVMAE': '9', 'CONSULTAS': '9'})
print(sinasc_edit['ESTCIVMAE'].isna().sum())
print(sinasc_edit_a['ESTCIVMAE'].isna().sum())
sinasc_edit_a.head(10)

315
0


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0


In [86]:
# 6) Substituir os valores faltantes da quantitativa (QTDFILVIVO) por zero.

sinasc_edit_b = sinasc_edit_a.fillna({'QTDFILVIVO': 0})
print(sinasc_edit_a['QTDFILVIVO'].isna().sum())
print(sinasc_edit_b['QTDFILVIVO'].isna().sum())
sinasc_edit_b.head(10)

1566
0


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0


In [97]:
"""
7) Já tratamos as colunas: 'ESTCIVMAE' 'QTDFILVIVO' 'CONSULTAS' 'APGAR5'
Trataremos as colunas: 'LOCNASC' 'IDADEMAE' 'ESCMAE' 'GESTACAO' 'GRAVIDEZ'
Faremos as seguintes substituições:
'LOCNASC' -> '9'
'IDADEMAE' -> média das idades como um número inteiro
'ESCMAE' -> 'Ignorado'
'GESTACAO' -> 'Ignorado'
'GRAVIDEZ' -> 'Ignorado'
"""
media_mae = int(sinasc_edit_b['IDADEMAE'].mean())

sinasc_edit_c = sinasc_edit_b.fillna({'LOCNASC': '9', 
                                    'IDADEMAE': media_mae,
                                    'ESCMAE': 'Ignorado',
                                    'GESTACAO': 'Ignorado',
                                    'GRAVIDEZ': 'Ignorado'})
print(sinasc_edit_b['ESCMAE'].isna().sum())
print(sinasc_edit_c['ESCMAE'].isna().sum())
sinasc_edit_c.head(10)

310
0


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0


In [109]:
"""8) Classificação Apgar:
Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.

Criar uma categorização calcular as frequências.
"""
sinasc_edit_c.loc[sinasc_edit_c['APGAR5'] >= 8, 'APGAR5_CAT'] = 'normal'
sinasc_edit_c.loc[(sinasc_edit_c['APGAR5'] >= 6) & (sinasc_edit_c['APGAR5'] <= 7), 'APGAR5_CAT'] = 'asfixia leve'
sinasc_edit_c.loc[(sinasc_edit_c['APGAR5'] >= 4) & (sinasc_edit_c['APGAR5'] <= 5), 'APGAR5_CAT'] = 'asfixia moderada'
sinasc_edit_c.loc[sinasc_edit_c['APGAR5'] <= 3, 'APGAR5_CAT'] = 'asfixia severa'
sinasc_edit_c.head(10)

print(sinasc_edit_c['APGAR5_CAT'].value_counts())
print(sinasc_edit_c['APGAR5_CAT'].value_counts(normalize=True))
sinasc_edit_c.head(10)

normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: APGAR5_CAT, dtype: int64
normal              0.982841
asfixia leve        0.011885
asfixia severa      0.002748
asfixia moderada    0.002526
Name: APGAR5_CAT, dtype: float64


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [119]:
# 9) Renomeiar as variáveis para que fiquem no snake case, com um underscore entre as palavras.

sinasc_edit_d = sinasc_edit_c.rename(columns = {'LOCNASC': 'loc_nasc', 
                                                'IDADEMAE': 'idade_mae', 
                                                'ESTCIVMAE': 'est_civ_mae', 
                                                'ESCMAE': 'esc_mae', 
                                                'QTDFILVIVO': 'qtd_fil_vivo', 
                                                'GESTACAO': 'gestacao', 
                                                'GRAVIDEZ': 'gravidez', 
                                                'CONSULTAS': 'consultas', 
                                                'APGAR5': 'apgar5', 
                                                'APGAR5_CAT': 'apgar5_cat'})
sinasc_edit_d.head(10)

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,apgar5_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
